In [2]:
from datasets import load_dataset

/Users/shilongcui/shilongjaycui/mental-health-insights/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
dataset = load_dataset('emotion')

In [12]:
import pandas as pd

pd.set_option('display.max_colwidth', None)  # Show full column width

In [21]:
from pandas import DataFrame

train_dataset = dataset['train']
train_df: DataFrame = dataset["train"].to_pandas()
train_df.sample(n=5)

,text,label
11762,i could only describe as feeling like there s something moving inside you it s not pleasant but it s nothing like true cramps impossible to describe unless you ve been poked from the inside out,1
8709,i recently learned that there is a very slight difference between empathetic and sympathetic in definition empathetic being able to actually feel the emotion and sympathetic being the ability to understand the emotion and i realized that there is also a fine line between the two in writing,2
1163,i feel so unimportant insignificant like im slipping through the gaps between his fingers and he doesnt care,0
255,i started the third block feeling hot and cold and tingly all at the same time knowing that i still had five hours of examination ahead of me having no idea if any of it would do any good,2
15777,im feeling kind of unwelcome,0


In [22]:
from typing import Dict

emotion_dict: Dict[int, str] = {
    0: 'sadness',
    1: 'joy',
    2: 'love',
    3: 'anger',
    4: 'fear',
    5: 'surprise',
}

train_df_copy = train_df.copy(deep=True)
train_df_copy['emotion'] = train_df_copy['label'].apply(lambda index: emotion_dict[index])
train_df_copy = train_df_copy[['text', 'emotion']]
train_df_copy

,text,emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplace i will know that it is still on the property,love
4,i am feeling grouchy,anger
...,...,...
15995,i just had a very brief time in the beanbag and i said to anna that i feel like i have been beaten up,sadness
15996,i am now turning and i feel pathetic that i am still waiting tables and subbing with a teaching degree,sadness
15997,i feel strong and good overall,joy
15998,i feel like this was such a rude comment and im glad that t,anger


In [33]:
from pandas import Series
X: Series = train_df['text']
y: Series = train_df['label']

print(f'X.shape: {X.shape}')
print(f'X: {X.tolist()[:5]}')

print(f'\ny.shape: {y.shape}')
print(f'y: {y.tolist()[:5]}')

X.shape: (16000,)
X: ['i didnt feel humiliated', 'i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake', 'im grabbing a minute to post i feel greedy wrong', 'i am ever feeling nostalgic about the fireplace i will know that it is still on the property', 'i am feeling grouchy']

y.shape: (16000,)
y: [0, 0, 3, 2, 3]


In [34]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

In [35]:
# Creating the pipeline
model = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', RandomForestClassifier())
])

In [36]:
model.fit(X, y)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf', RandomForestClassifier())])

In [38]:
from sklearn.metrics import accuracy_score, classification_report

print(dataset.column_names)

# For a classification task
X_test = dataset['test']['text']
y_test = dataset['test']['label']

predictions = model.predict(X=X_test)  # predict the actual class
print("Accuracy:", accuracy_score(y_true=y_test, y_pred=predictions))
print(classification_report(y_test, predictions))

{'train': ['text', 'label'], 'validation': ['text', 'label'], 'test': ['text', 'label']}
Accuracy: 0.8665
              precision    recall  f1-score   support

           0       0.94      0.90      0.92       581
           1       0.84      0.94      0.88       695
           2       0.81      0.64      0.71       159
           3       0.88      0.84      0.86       275
           4       0.85      0.86      0.85       224
           5       0.71      0.53      0.61        66

    accuracy                           0.87      2000
   macro avg       0.84      0.78      0.81      2000
weighted avg       0.87      0.87      0.86      2000



Create a precision-recall curve.

In [39]:
# Get predicted probabilities for the positive class
y_scores = model.predict_proba(X=X_test)[:, 1]  # predict the class probabilities
y_scores

array([0.21 , 0.09 , 0.22 , ..., 0.74 , 0.685, 0.13 ])

In [42]:
from sklearn.metrics import precision_recall_curve

# Compute precision-recall curve values
try:
    precision, recall, _ = precision_recall_curve(y_true=y_test, probas_pred=y_scores)
except ValueError:
    print("`precision_recall_curve` is designed for binary classification problems.")

`precision_recall_curve` is designed for binary classification problems.
